In [1]:
import sys

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

In [2]:
# Test Pareto Optimizer

from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer
from robyn.debug.data_mapper import (
    import_output_models,
    import_input_collect,
    load_data_from_json,
)

# /Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/debug

2025-03-11 07:30:42,162 - robyn - INFO - Logging is set up to console only.
/Users/yijuilee/robynpy_release_reviews/Robyn/.pyvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data from JSON
inputCollect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_InputCollect.json"
)
outputModel = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_OutputModels.json"
)
input_collect = import_input_collect(inputCollect)
output_models = import_output_models(outputModel)

Raw hyperparameters data: {'facebook_S_alphas': [0.5, 3], 'facebook_S_gammas': [0.3, 1], 'facebook_S_thetas': [0, 0.3], 'print_S_alphas': [0.5, 3], 'print_S_gammas': [0.3, 1], 'print_S_thetas': [0.1, 0.4], 'tv_S_alphas': [0.5, 3], 'tv_S_gammas': [0.3, 1], 'tv_S_thetas': [0.3, 0.8], 'search_S_alphas': [0.5, 3], 'search_S_gammas': [0.3, 1], 'search_S_thetas': [0, 0.3], 'ooh_S_alphas': [0.5, 3], 'ooh_S_gammas': [0.3, 1], 'ooh_S_thetas': [0.1, 0.4], 'newsletter_alphas': [0.5, 3], 'newsletter_gammas': [0.3, 1], 'newsletter_thetas': [0.1, 0.4], 'train_size': [0.5, 0.8]}
Debug: R output data keys: ['trial1', 'trial2', 'trial3', 'trial4', 'trial5']
Data for trial1: Keys = ['resultCollect', 'hyperBoundNG', 'hyperBoundFixed']
Debug: resultCollect keys for trial1: ['resultHypParam', 'xDecompAgg', 'liftCalibration', 'decompSpendDist', 'iter', 'elapsed.min']
Debug: Sample resultHypParam for trial1: [{'facebook_S_alphas': 1.98202923325, 'facebook_S_gammas': 0.58424338159, 'facebook_S_thetas': 0.0907

In [4]:
mmm_data = input_collect["mmm_data"]
# display(mmm_data.data.head())
# Display Model Outputs
print(mmm_data.mmmdata_spec.rolling_window_start_which)
print(mmm_data.mmmdata_spec.rolling_window_end_which)
print(mmm_data.mmmdata_spec.window_start)
print(mmm_data.mmmdata_spec.window_end)

output_models = output_models
# display((model_outputs.trials[0].result_hyp_param))

hyperparameters = input_collect["hyperparameters"]
# display(hyperparameters)

featurized_mmm_data = input_collect["featurized_mmm_data"]

holidays_data = input_collect["holidays_data"]

6
162


In [5]:
# Add after model training
print("Model Output Summary:")
print(f"Number of trials: {len(output_models.trials)}")
print(
    f"Average models per trial: {len(output_models.all_result_hyp_param) / len(output_models.trials)}"
)
print(
    f"Total unique models: {len(output_models.all_result_hyp_param['sol_id'].unique())}"
)

print("\nMetrics Distribution:")
metrics_df = output_models.all_result_hyp_param[["nrmse", "decomp.rssd", "mape"]]
print(metrics_df.describe())

# Additional validation to debug model output
print("\nColumns in result_hyp_param:")
print(output_models.all_result_hyp_param.columns.tolist())

print("\nSample rows of metrics:")
print(
    output_models.all_result_hyp_param[
        ["sol_id", "nrmse", "decomp.rssd", "mape"]
    ].head()
)

# Show shape of result dataframes
print("\nDataFrame Shapes:")
print(f"result_hyp_param: {output_models.all_result_hyp_param.shape}")
print(f"x_decomp_agg: {output_models.all_x_decomp_agg.shape}")
print(f"decomp_spend_dist: {output_models.all_decomp_spend_dist.shape}")

Model Output Summary:
Number of trials: 5
Average models per trial: 2007.0
Total unique models: 10035

Metrics Distribution:
              nrmse   decomp.rssd          mape
count  10035.000000  10035.000000  10035.000000
mean       0.285779      0.628863    176.847461
std        0.067437      0.083541    167.500223
min        0.085069      0.369959      0.812119
25%        0.242605      0.565598     87.377313
50%        0.264548      0.631424    115.026803
75%        0.301364      0.696620    182.792278
max        0.521505      0.873183   1129.896150

Columns in result_hyp_param:
['facebook_S_alphas', 'facebook_S_gammas', 'facebook_S_thetas', 'newsletter_alphas', 'newsletter_gammas', 'newsletter_thetas', 'ooh_S_alphas', 'ooh_S_gammas', 'ooh_S_thetas', 'print_S_alphas', 'print_S_gammas', 'print_S_thetas', 'search_S_alphas', 'search_S_gammas', 'search_S_thetas', 'tv_S_alphas', 'tv_S_gammas', 'tv_S_thetas', 'train_size', 'rsq_train', 'rsq_val', 'rsq_test', 'nrmse_train', 'nrmse_val', 'nrm

In [6]:
# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(
    mmm_data, output_models, hyperparameters, featurized_mmm_data, holidays_data
)

In [7]:
# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=100)

2025-03-11 07:30:52 [INFO] Starting Pareto optimization
2025-03-11 07:30:52,610 - robyn.modeling.pareto.data_aggregator - INFO - Starting model data aggregation
2025-03-11 07:30:52 [INFO] Computing Pareto fronts
2025-03-11 07:30:53 [INFO] Pareto front computation completed
2025-03-11 07:30:53 [INFO] Preparing Pareto data
2025-03-11 07:30:53 [INFO] Number of Pareto-optimal solutions found: 637
2025-03-11 07:30:53 [INFO] Selected 10 Pareto-fronts containing 102 candidates
2025-03-11 07:30:53 [INFO] Selected Pareto fronts: 11
2025-03-11 07:30:53 [INFO] Filtering data for selected Pareto fronts...
2025-03-11 07:30:53 [INFO] Pareto data preparation completed
2025-03-11 07:30:53,190 - robyn.modeling.pareto.response_curve - INFO - Calculating response curves for 510 models' media variables...
Processing rows: 100%|██████████| 510/510 [00:03<00:00, 137.05it/s] 
2025-03-11 07:30:58,298 - robyn.modeling.pareto.response_curve - INFO - Successfully processed 510 response curves
2025-03-11 07:30:58

KeyboardInterrupt: 

In [ ]:
# 5. Check results
print("Pareto Optimization Results:")
print(
    f"Number of Pareto fronts: {len(pareto_result.pareto_solutions)} - {pareto_result.pareto_solutions}"
)
print(
    f"MediaVecCollect: {pareto_result.media_vec_collect.shape, pareto_result.media_vec_collect}"
)
print("\Hyper parameter solutions:")
print(pareto_result.result_hyp_param)

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg)
print("\result Calibration:")
print(pareto_result.result_calibration)
print("\nx Decomp Vec Collect:")
print(pareto_result.x_decomp_vec_collect.shape, pareto_result.x_decomp_vec_collect)
print("\nCarryover percentage all:")
print(pareto_result.df_caov_pct_all.shape, pareto_result.df_caov_pct_all)
print("\Plot Data Collected")
# print("NUMBER OF PLOTS Data collected for:", len(pareto_result.plot_data_collect["2_4_8"]))
# print("Plot data for solid 3_206_6", pareto_result.plot_data_collect["2_4_8"])

# 6. Validate logic
assert pareto_result.pareto_fronts == "auto" or isinstance(
    pareto_result.pareto_fronts, int
), "Invalid pareto_fronts value"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")

In [ ]:
print(pareto_result.x_decomp_agg[pareto_result.x_decomp_agg["sol_id"] == "5_221_9"])

# Clustering

In [ ]:
from robyn.modeling.clustering.clustering_config import ClusteringConfig, ClusterBy
from robyn.modeling.clustering.cluster_builder import ClusterBuilder
from robyn.data.entities.enums import DependentVarType
import plotly.io as pio

pio.renderers.default = "iframe"

cluster_configs = ClusteringConfig(
    dep_var_type=DependentVarType(mmm_data.mmmdata_spec.dep_var_type),
    cluster_by=ClusterBy.HYPERPARAMETERS,
    max_clusters=10,
    min_clusters=3,
    weights=[1.0, 1.0, 1.0],
)

cluster_builder = ClusterBuilder(pareto_result=pareto_result)

In [ ]:
print(mmm_data.mmmdata_spec.rolling_window_start_which)
print(mmm_data.mmmdata_spec.rolling_window_end_which)

In [ ]:
cluster_results = cluster_builder.cluster_models(cluster_configs)

In [ ]:
# Cluster result Validations (graphs)
print(cluster_results.top_solutions["sol_id"])
cluster_results.wss

In [ ]:
cluster_results.plots.top_solutions_errors_plot

In [ ]:
cluster_results.plots.top_solutions_rois_plot

In [ ]:
cluster_results.cluster_ci.clusters_confidence_interval_plot

### Process Pareto Clustered Results for Allocator

In [16]:
from robyn.modeling.pareto.pareto_utils import ParetoUtils

utils = ParetoUtils()
filtered_pareto_results = utils.process_pareto_clustered_results(
    pareto_result,
    clustered_result=cluster_results,
    ran_cluster=True,
    ran_calibration=False,
)

# One Pager Results

In [ ]:
from robyn.data.entities.enums import AdstockType
from robyn.reporting.onepager_reporting import OnePager
from robyn.visualization.pareto_visualizer import ParetoVisualizer


visualizer = OnePager(
    pareto_result=filtered_pareto_results,
    clustered_result=cluster_results,
    hyperparameter=hyperparameters,
    mmm_data=mmm_data,
    holidays_data=holidays_data,
)
visualizer.generate_one_pager(top_pareto=True)

In [ ]:
visualizer = ParetoVisualizer(
    pareto_result=filtered_pareto_results,
    hyperparameter=hyperparameters,
    mmm_data=mmm_data,
    holiday_data=holidays_data,
    featurized_mmm_data=featurized_mmm_data,
    unfiltered_pareto_result=pareto_result,
    model_outputs=output_models,
)

visualizer.plot_all(True)

# Allocator